# Model Implementation

In [24]:
!pip install pyodbc

## Data Loading

In [25]:
import pyodbc
import pandas as pd

# Database credentials
server = 'techentjan.database.windows.net'
database = 'QAECECRM_jan'
username = 'azureml_user'
password = 'ElevateLivePro12!'

# Connection string
connection_string = f"DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}"

# Establish connection
conn = pyodbc.connect(connection_string)

# Fetch pre-merged data
query = "SELECT * FROM dbo.Presenter_Artist_Interactions;"

# Load data into Pandas DataFrame
df = pd.read_sql(query, conn)

# Close connection
conn.close()

In [26]:
df.head()

,PresenterId,ArtistId,FeedbackScore,Implicit_Score
0,0,296,5.0,0
1,0,687,5.0,0
2,0,6542,4.0,0
3,0,7426,5.0,0
4,0,13166,5.0,0


In [27]:
# Display dataframe info
print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 277961 entries, 0 to 277960
Data columns (total 4 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   PresenterId     277961 non-null  int64  
 1   ArtistId        277961 non-null  int64  
 2   FeedbackScore   277961 non-null  float64
 3   Implicit_Score  277961 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 8.5 MB
None
   PresenterId  ArtistId  FeedbackScore  Implicit_Score
0            0       296            5.0               0
1            0       687            5.0               0
2            0      6542            4.0               0
3            0      7426            5.0               0
4            0     13166            5.0               0


No missing values were found

## Data Preprocessing

In [28]:
import pandas as pd

# Check for duplicate records
duplicate_count = df.duplicated(subset=['PresenterId', 'ArtistId']).sum()
print(f"Number of duplicate (PresenterId, ArtistId) pairs: {duplicate_count}")

# Check for invalid IDs (e.g., PresenterId = 0)
invalid_presenter_count = (df["PresenterId"] == 0).sum()
print(f"Number of records with PresenterId = 0: {invalid_presenter_count}")

# Describe numerical columns
print(df.describe())

# Check unique values for Implicit_Score
print("Unique values in Implicit_Score:", df["Implicit_Score"].unique())

# Ensure FeedbackScore is within expected range
print("FeedbackScore range:", df["FeedbackScore"].min(), "to", df["FeedbackScore"].max())

Number of duplicate (PresenterId, ArtistId) pairs: 0
Number of records with PresenterId = 0: 33
        PresenterId       ArtistId  FeedbackScore  Implicit_Score
count  2.779610e+05  277961.000000  277961.000000   277961.000000
mean   7.003728e+06   21848.128482       0.826339        0.829998
std    3.829318e+06   26440.339041       1.850712        0.375635
min    0.000000e+00       2.000000       0.000000        0.000000
25%    5.275100e+06    4223.000000       0.000000        1.000000
50%    8.610393e+06   16632.000000       0.000000        1.000000
75%    1.000908e+07   30362.000000       0.000000        1.000000
max    1.002129e+07  104194.000000       5.000000        1.000000
Unique values in Implicit_Score: [0 1]
FeedbackScore range: 0.0 to 5.0


- No duplicate
- 33 records have PresenterId = 0. We may need to filter them out.
- FeedbackScore is heavily skewed towards 0
    - Mean 0.82, Std Dev 1.85, Max 5.0.
    - Many zero values (potential missing explicit feedback).
- Implicit_Score values are fine.


In [29]:
from sklearn.preprocessing import MinMaxScaler

# Remove invalid PresenterId 
df = df[df["PresenterId"] != 0]
print(f"Remaining records after removing PresenterId=0: {len(df)}")

# Normalize FeedbackScore
scaler = MinMaxScaler(feature_range=(0, 1))  # Scale to range [0,1]
df["FeedbackScore_Norm"] = scaler.fit_transform(df[["FeedbackScore"]])
print("FeedbackScore successfully normalized.")

# Show dataset summary
print(df.describe())

# Display first few rows
print(df.head())


Remaining records after removing PresenterId=0: 277928
FeedbackScore successfully normalized.
        PresenterId       ArtistId  FeedbackScore  Implicit_Score  \
count  2.779280e+05  277928.000000  277928.000000   277928.000000   
mean   7.004560e+06   21846.638133       0.825872        0.830096   
std    3.828785e+06   26438.716623       1.850306        0.375549   
min    1.000000e+00       2.000000       0.000000        0.000000   
25%    5.275250e+06    4223.000000       0.000000        1.000000   
50%    8.610401e+06   16632.000000       0.000000        1.000000   
75%    1.000908e+07   30362.000000       0.000000        1.000000   
max    1.002129e+07  104194.000000       5.000000        1.000000   

       FeedbackScore_Norm  
count       277928.000000  
mean             0.165174  
std              0.370061  
min              0.000000  
25%              0.000000  
50%              0.000000  
75%              0.000000  
max              1.000000  
    PresenterId  ArtistId  Feedb

## User-Item Interaction Matrix for SVD

In [30]:
import numpy as np
from scipy.sparse import coo_matrix

# Define unique presenter and artist mappings
presenter_ids = df["PresenterId"].unique()
artist_ids = df["ArtistId"].unique()

# Create mappings for index positions
presenter_to_index = {pid: idx for idx, pid in enumerate(presenter_ids)}
artist_to_index = {aid: idx for idx, aid in enumerate(artist_ids)}

# Map presenter and artist IDs to matrix indices
row_indices = df["PresenterId"].map(presenter_to_index)
col_indices = df["ArtistId"].map(artist_to_index)

# Use FeedbackScore_Norm; if missing, use Implicit_Score
ratings = df["FeedbackScore_Norm"].fillna(df["Implicit_Score"]).values

# Construct sparse matrix
interaction_sparse = coo_matrix((ratings, (row_indices, col_indices)), 
                                shape=(len(presenter_ids), len(artist_ids)))

# Display matrix shape
print(f"Sparse User-Item Matrix Shape: {interaction_sparse.shape}")

Sparse User-Item Matrix Shape: (74006, 8365)


## Train the SVD Collaborative Filtering Model

In [31]:
from sklearn.decomposition import TruncatedSVD

# Define the number of latent features
n_components = 800  # You can tune this value

# Convert the sparse matrix to CSR format for efficiency
interaction_sparse_csr = interaction_sparse.tocsr()

# Train SVD model
svd = TruncatedSVD(n_components=n_components, random_state=42)
latent_matrix = svd.fit_transform(interaction_sparse_csr)

# Display explained variance (helps to choose `n_components`)
print(f"Explained variance ratio: {svd.explained_variance_ratio_.sum():.4f}")

Explained variance ratio: 0.8360


In [32]:
# Reconstruct the User-Item Interaction Matrix using SVD components
reconstructed_matrix = np.dot(latent_matrix, svd.components_)

# Convert back to original user-item index mapping
predicted_df = pd.DataFrame(reconstructed_matrix, index=presenter_ids, columns=artist_ids)

# Function to get top N artist recommendations for a given presenter
def get_top_recommendations(presenter_id, top_n=10):
    if presenter_id not in predicted_df.index:
        return f"Presenter {presenter_id} not found in dataset."

    # Get predictions for this presenter
    predictions = predicted_df.loc[presenter_id]

    # Sort artists by predicted score in descending order
    top_artists = predictions.sort_values(ascending=False).head(top_n)

    return top_artists

# Get top 10 recommended artists for a sample PresenterId 
sample_presenter = presenter_ids[0]  # Take the first presenter
recommendations = get_top_recommendations(sample_presenter, top_n=10)

# Display recommendations
print(f"Top 10 Artist Recommendations for Presenter {sample_presenter}:")
print(recommendations)

Top 10 Artist Recommendations for Presenter 1:
3247      4.296810e-07
19143     4.087347e-07
31682     3.977467e-07
10510     3.848672e-07
3758      3.603463e-07
4828      3.386555e-07
6628      3.373267e-07
7431      3.016719e-07
101169    2.920683e-07
191       2.920572e-07
Name: 1, dtype: float64


Recommendation scores are extremely low (e-07 range).